In [4]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import argparse
import imutils
import time
import dlib
import cv2
from keras.models import load_model
import requests as req
import concurrent.futures

In [5]:
model = load_model('./model.h5')

AttributeError: 'str' object has no attribute 'decode'

In [3]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 150, 150, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 150, 150, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization_2[0][0]            
____________________________________________________________________________________________

In [4]:
def sound_alarm(path):
    playsound.playsound(path)
    record_alarm()

In [5]:
def record_alarm():
    url = 'https://webtech-lab-jominjose.000webhostapp.com/sleepywheels/alarm.php'
    resp = req.post(url, data = {'alarm': 'ALARM'})
    if resp != None:
        print('Alarm recorded')

In [6]:
def record_yawn():
    url = 'https://webtech-lab-jominjose.000webhostapp.com/sleepywheels/yawn.php'
    resp = req.post(url, data = {'yawn': 'YAWN'})
    if resp != None:
        print('Yawn recorded')

In [7]:
def effnet(nn_frame):
    global COUNTER
    p = model.predict(nn_frame.reshape(-1, 150, 150, 3))
    return p

In [8]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

In [9]:
def mouth_aspect_ratio(mouthInline):
    # compute the euclidean distances between the five sets of
    # vertical mouth outline landmarks (x, y)-coordinates
    A = dist.euclidean(mouthInline[1], mouthInline[7])
    B = dist.euclidean(mouthInline[2], mouthInline[6])
    C = dist.euclidean(mouthInline[3], mouthInline[5])
    # compute the euclidean distance between the horizontal
    # mouth outline landmark (x, y)-coordinates
    D = dist.euclidean(mouthInline[0], mouthInline[4])
    # compute the eye aspect ratio
    mar = (A+B+C) / (3.0 * D)
    # return the eye aspect ratio
    return mar

In [10]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.26
PROB_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 15
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

In [11]:
CNN_COUNTER = 0
CNN_CONSEC_FRAMES = 15

In [12]:
# define two constants, one for the mouth aspect ratio to indicate
# yawn and then a second constant for the number of consecutive
# frames the mouth must be above the threshold to record a
# yawn
MOUTH_AR_THRESH = 0.05
MOUTH_AR_CONSEC_FRAMES = 20
# initialize the yawn frame counter
MCOUNTER = 0

In [13]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./Pretrained Models/shape_predictor_68_face_landmarks.dat")
print("Loaded facial landmark predictor successfully")

Loaded facial landmark predictor successfully


In [14]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [15]:
# Initialize variable for number of yawns
yawns = 0

In [17]:
# start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(src=0).start()
time.sleep(1.0)
# loop over frames from the video stream
while True:
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
    frame = vs.read()
    
    nn_frame = cv2.resize(frame, (150,150), interpolation = cv2.INTER_AREA)
    
    #pass the image to neural network on new thread
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         future = executor.submit(effnet, nn_frame)
#         prob = future.result()
#         cv2.putText(frame, "CNN: {:.2f}".format(prob[0][0]), (300, 40),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
    prob = effnet(nn_frame)
    cv2.putText(frame, "CNN: {:.2f}".format(prob[0][0]), (10, 45),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    if prob < PROB_THRESH:
        CNN_COUNTER += 1
    else:
        CNN_COUNTER = 0
    
    if CNN_COUNTER >= CNN_CONSEC_FRAMES:
            # if the alarm is not on, turn it on
            if not ALARM_ON:
                ALARM_ON = True
                t = Thread(target=sound_alarm,
                    args=("./Sounds/alarm.wav",))
                t.deamon = True
                t.start()
            # draw an alarm on the frame
            cv2.putText(frame, "Wake Up!", (10, 75),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        
        
        
        
        
        
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces in the grayscale frame
    rects = detector(gray, 0)
    
    
    
    
    
    
    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        # extract the left and right eye coordinates
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        # compute the eye aspect ratio for both eyes
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        
        # coordinates of mouth and figuring out the mouth outer and inner lining
#         mouth = shape[mStart:mEnd]
        mouthOutline = shape[mStart:mStart+12]
        mouthInline = shape[60:]
        # find mouth aspect ratio
        mar = mouth_aspect_ratio(mouthInline)
        
        
        
        
        
        
        
        
        
        # compute the convex hull for the left, right eye and mouth's inner and outer outlines
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        mouthOutlineHull = cv2.convexHull(mouthOutline)
        mouthInlineHull = cv2.convexHull(mouthInline)
        # visualize each of the eyes and mouth outlines
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [mouthOutlineHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [mouthInlineHull], -1, (0, 255, 0), 1)
        
        
        
        
        
        
        
        
        
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            
#         # otherwise, the eye aspect ratio is not below the blink
#         # threshold, so reset the counter and alarm
        else:
            COUNTER = 0
            ALARM_ON = False
        
        
        
        
        
        
        
        
        # if the eyes were closed for a sufficient number of frames
        # then sound the alarm
        if COUNTER >= EYE_AR_CONSEC_FRAMES:
            # if the alarm is not on, turn it on
            if not ALARM_ON:
                ALARM_ON = True
                t = Thread(target=sound_alarm,
                    args=("./Sounds/alarm.wav",))
                t.deamon = True
                t.start()
            # draw an alarm on the frame
            cv2.putText(frame, "Wake Up!", (320, 55),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and sleepy frame counters
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (320, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        
        
        
        
        
        
            
        # check to see if the mouth aspect ratio is above the yawn
        # threshold, and if so, increment the yawn frame counter
        if mar > MOUTH_AR_THRESH:
            MCOUNTER += 1
            # if the mouth was wide open for a sufficient number of frames
            # then sound the alarm
            if MCOUNTER == MOUTH_AR_CONSEC_FRAMES:
                # draw a yawn notification on the frame
                yawns += 1
                cv2.putText(frame, "Yawn recorded!", (100, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                record_yawn()
        # otherwise, the mouth aspect ratio is not above the yawn
        # threshold, so reset the counter
        else:
            MCOUNTER = 0
            
        # draw the computed mouth aspect ratio on the frame to help
        # with debugging and setting the correct mouth aspect ratio
        # thresholds and yawn frame counters
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (320, 320),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # display number of yawns recorded
        cv2.putText(frame, "Yawns recorded: "+str(yawns), (10, 320),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    
    
    
    
    
    
    
    # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream thread...
Alarm recorded
Yawn recorded
Alarm recorded
